In [1]:
import sys
import numpy as np
import numpy.typing as npt
import time as time
from collections import deque
from HashArray import HashableNdarray

In [14]:
# Global params
n = 3  # size of matrix
p = 7  # Z/pZ
# m = int(1E1)    # number of matrices to generate


# Set which generates SL(n, Z) (n = 3). Use these as edges in the graph
A_3 = (np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])).view(HashableNdarray)
Ai_3 = (np.array([[1, -1, 0], [0, 1, 0], [0, 0, 1]])).view(HashableNdarray)
# Ai_3 = np.array([[1,2,0],[0,1,0],[0,0,1]])
B_3 = (np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])).view(HashableNdarray)
Bi_3 = (np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])).view(HashableNdarray)
edges_3 = [A_3, Ai_3, B_3, Bi_3]

A_2 = (np.array([[1, 1], [0, 1]])).view(HashableNdarray)
Ai_2 = (np.array([[1, -1], [0, 1]])).view(HashableNdarray)
B_2 = (np.array([[0, 1], [-1, 0]])).view(HashableNdarray)
Bi_2 = (np.array([[0, -1], [1, 0]])).view(HashableNdarray)
edges_2 = [A_2, Ai_2, B_2, Bi_2]

In [79]:
def generate_set_bfs(s: set[npt.NDArray[np.int_]] = set(), n: int = 3, p: int = 11):
    if n == 2:
        edges = edges_2
    if n == 3:
        edges = edges_3

    queue = deque(edges)
    while queue:
        x = queue.popleft()
        for e in edges:
            Xe = np.matmul(x, e)
            Xe = Xe % p
            if Xe not in s:
                s.add(Xe)
                queue.append(Xe)

    return s

In [12]:
def test_generate_set_bfs(s: set[npt.NDArray[np.int_]] = set(), n: int = 3, p: int = 11):
    if n == 2:
        edges = edges_2
    if n == 3:
        edges = edges_3

    queue = deque(edges)
    while queue:
        x = queue.popleft()
        Xes = np.matmul(x, edges)
        Xes = Xes % p
        for Xe in Xes:
            if Xe not in s:
                s.add(Xe)
                queue.append(Xe)

    return s

In [ ]:
%timeit generate_set_bfs(n=2, p=7)

In [ ]:
%timeit test_generate_set_bfs(n=2, p=7)

In [2]:
%load_ext line_profiler

In [76]:
%lprun -f generate_set_bfs generate_set_bfs(n=3, p=5)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 8.94241 s
File: /var/folders/gk/nd3wqtqx291b3hj52v6ll0qw0000gn/T/ipykernel_19630/2997984984.py
Function: generate_set_bfs at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def generate_set_bfs(s: set[npt.NDArray[np.int_]] = set(), n: int = 3, p: int = 11):
     2         1          1.0      1.0      0.0      if n == 2:
     3                                                   edges = edges_2
     4         1          0.0      0.0      0.0      if n == 3:
     5         1          0.0      0.0      0.0          edges = edges_3
     6                                           
     7         1          1.0      1.0      0.0      queue = deque(edges)
     8    286055      52789.0      0.2      0.6      while queue:
     9    286055      66790.0      0.2      0.7          x = queue.popleft()
    10   1430271     289236.0      0.2      3.2          for e in edges:
    11   1144217    177

In [ ]:
%lprun -f test_generate_set_bfs test_generate_set_bfs(n=3, p=5)

In [ ]:
test_A = np.asarray(A_2)
print(tuple(test_A))
a = set()
a.add(tuple(test_A))

In [3]:
from numba import jit

@jit(nopython=True)
def is_not_in_set(Xe_tup: tuple, s: set) -> bool:
    return Xe_tup not in s

In [16]:
def tuple_generate_set_bfs(s: set[tuple] = set(), n: int = 3, p: int = 11):
    if n == 2:
        A_2 = np.array([[1, 1], [0, 1]])
        Ai_2 = np.array([[1, -1], [0, 1]])
        B_2 = np.array([[0, 1], [-1, 0]])
        Bi_2 = np.array([[0, -1], [1, 0]])
        edges_2 = np.array([A_2, Ai_2, B_2, Bi_2])
        edges = edges_2
        # s.add((0 for i in range(4)))
    if n == 3:
        A_3 = np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])
        Ai_3 = np.array([[1, -1, 0], [0, 1, 0], [0, 0, 1]])
        # Ai_3 = np.array([[1,2,0],[0,1,0],[0,0,1]])
        B_3 = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        Bi_3 = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
        edges_3 = np.array([A_3, Ai_3, B_3, Bi_3])

        edges = edges_3
        # s.add(tuple([0 for i in range(9)]))

    queue = deque(edges)

    while queue:
        x = queue.popleft()
        Xes = np.matmul(x, edges)
        Xes = Xes % p
        for Xe in Xes:
            Xe_tup = tuple(np.ravel(Xe))
            if Xe_tup not in s:
                s.add(Xe_tup)
                queue.append(Xe)

    return s

In [18]:
%lprun -f tuple_generate_set_bfs tuple_generate_set_bfs(set(), n=2, p=11)

Timer unit: 1e-06 s

Total time: 0.020867 s
File: /var/folders/gk/nd3wqtqx291b3hj52v6ll0qw0000gn/T/ipykernel_24679/1293292961.py
Function: tuple_generate_set_bfs at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def tuple_generate_set_bfs(s: set[tuple] = set(), n: int = 3, p: int = 11):
     2         1          2.0      2.0      0.0      if n == 2:
     3         1         13.0     13.0      0.1          A_2 = np.array([[1, 1], [0, 1]])
     4         1          2.0      2.0      0.0          Ai_2 = np.array([[1, -1], [0, 1]])
     5         1          2.0      2.0      0.0          B_2 = np.array([[0, 1], [-1, 0]])
     6         1          2.0      2.0      0.0          Bi_2 = np.array([[0, -1], [1, 0]])
     7         1         13.0     13.0      0.1          edges_2 = np.array([A_2, Ai_2, B_2, Bi_2])
     8         1          0.0      0.0      0.0          edges = edges_2
     9                               

In [15]:
%lprun -f test_generate_set_bfs test_generate_set_bfs(set(), n=2, p=7)


Timer unit: 1e-06 s

Total time: 0.012878 s
File: /var/folders/gk/nd3wqtqx291b3hj52v6ll0qw0000gn/T/ipykernel_24679/278545838.py
Function: test_generate_set_bfs at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def test_generate_set_bfs(s: set[npt.NDArray[np.int_]] = set(), n: int = 3, p: int = 11):
     2         1          1.0      1.0      0.0      if n == 2:
     3         1          0.0      0.0      0.0          edges = edges_2
     4         1          0.0      0.0      0.0      if n == 3:
     5                                                   edges = edges_3
     6                                           
     7         1          1.0      1.0      0.0      queue = deque(edges)
     8       341         90.0      0.3      0.7      while queue:
     9       340        110.0      0.3      0.9          x = queue.popleft()
    10       340       2604.0      7.7     20.2          Xes = np.matmul(x, edges)
   